In [25]:
from quoptuna.backend.data_processing.prepare import DataPreparation


In [26]:
!pip install matminer -q

/opt/homebrew/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [27]:
# from matminer.datasets import load_dataset
# from matminer.featurizers.conversions import StructureToComposition
# from matminer.featurizers.composition import ElementProperty
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report

# # Load a dataset from Matminer
# # Example: Load the 'matbench_jdft2d' dataset and assume binary classification on some property
# df = load_dataset('matbench_jdft2d')

# # Preprocess the data: Convert structure objects to composition objects
# df = StructureToComposition().featurize_dataframe(df, 'structure')

# # Featurize the data: Use ElementProperty to generate features based on composition
# ep_feat = ElementProperty.from_preset(preset_name="magpie")
# df = ep_feat.featurize_dataframe(df, col_id='composition', ignore_errors=True)

# # Define the target variable for binary classification
# # For demonstration, we'll create a binary target based on a threshold (e.g., band gap > 1 eV)
# df['target'] = (df['band_gap'] > 1.0).astype(int)

# # Drop any rows with NaN values that might have resulted from featurization errors
# df.dropna(inplace=True)

In [28]:
from matminer.datasets import load_dataset

# Load the 'matbench_glass' dataset
df = load_dataset('matbench_glass')

# Check the first few rows and columns of the dataset
print(df.head())
print(df.columns)


   composition    gfa
0           Al  False
1     Al(NiB)2   True
2  Al10Co21B19   True
3  Al10Co23B17   True
4  Al10Co27B13   True
Index(['composition', 'gfa'], dtype='object')


In [29]:
from matminer.datasets import load_dataset
from matminer.featurizers.composition import ElementFraction
from pymatgen.core.composition import Composition
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the 'matbench_glass' dataset
df = load_dataset('matbench_glass')

# Convert composition strings to Composition objects
df['composition'] = df['composition'].apply(Composition)

# Initialize the ElementFraction featurizer
element_fraction = ElementFraction()

# Featurize the composition column, ignoring errors
df = element_fraction.featurize_dataframe(df, 'composition', ignore_errors=True)

# Remove any rows with NaN values that might have resulted from featurization errors
df = df.dropna()

ElementFraction: 100%|██████████| 5680/5680 [01:40<00:00, 56.52it/s] 


In [30]:
df.head()

,composition,gfa,H,He,Li,Be,B,C,N,O,...,Mt,Ds,Rg,Cn,Nh,Fl,Mc,Lv,Ts,Og
0,(Al),False,0,0,0,0.0,0.00,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,"(Al, Ni, B)",True,0,0,0,0.0,0.40,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,"(Al, Co, B)",True,0,0,0,0.0,0.38,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,"(Al, Co, B)",True,0,0,0,0.0,0.34,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,"(Al, Co, B)",True,0,0,0,0.0,0.26,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
import pandas as pd
import os

def mock_csv_data(data, tmp_path, file_name=None):

    dataframe = pd.DataFrame(data)
    file_name = file_name if file_name else "mock_csv"
    file_path = os.path.join(tmp_path, f"{file_name}.csv")
    dataframe.to_csv(file_path, index=False)
    return file_path

In [32]:
file_path = mock_csv_data(df,tmp_path="data", file_name="matbench_glass")

In [33]:
df.columns

Index(['composition', 'gfa', 'H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O',
       ...
       'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og'],
      dtype='object', length=120)

In [34]:
# x_cols = list(df.columns.difference(['composition', 'gfa']))
x_cols=element_fraction.feature_labels()

In [35]:
# x_cols

In [36]:
data_prep = DataPreparation(file_path=file_path,
                                x_cols=x_cols,
                                y_col="gfa")

In [37]:
data_prep

In [38]:
x, y = data_prep.select_columns()
x_train, x_test, y_train, y_test = data_prep.prepare_data()
data_dict = {
                "train_x": x_train,
                "test_x": x_test,
                "train_y": y_train,
                "test_y": y_test,
            }

In [39]:
y_test

array([-1,  1, -1, ..., -1, -1, -1])

In [40]:
from quoptuna import Optimizer

In [41]:
file_name="matbench_glass"

In [42]:
optimizer = Optimizer(db_name=file_name, study_name=file_name,data=data_dict)

: 

In [43]:
study, best_trials = optimizer.optimize(n_trials=100)

[I 2024-11-21 08:55:26,827] A new study created in RDB with name: matbench_glass


[I 2024-11-21 08:55:27,085] Trial 0 finished with value: 0.47717231222385864 and parameters: {'max_vmap': 1, 'batch_size': 32, 'learning_rate': 0.01, 'n_input_copies': 1, 'n_layers': 1, 'observable_type': 'full', 'repeats': 1, 'C': 0.1, 'gamma_factor': 0.1, 'trotter_steps': 1, 't': 1.0, 'n_qfeatures': 'half', 'n_episodes': 500, 'visible_qubits': 'full', 'temperature': 10, 'encoding_layers': 3, 'degree': 3, 'n_qchannels': 10, 'qkernel_shape': 3, 'kernel_shape': 5, 'filter_name': 'edge_detect', 'gamma': 0.1, 'alpha': 0.01, 'hidden_layer_sizes': '[100,)', 'eta0': 1, 'model_type': 'SVClinear'}. Best is trial 0 with value: 0.47717231222385864.
/Users/edwinjose/Documents/GitHub/quoptuna/.venv/lib/python3.12/site-packages/pennylane/devices/default_qubit_jax.py:173: PennyLaneDeprecationWarning: Use of 'default.qubit.jax' is deprecated. Instead, use 'default.qubit', which supports backpropagation. If you experience issues, reach out to the PennyLane team on the discussion forum: https://discuss